In [2]:
!pip install ibm_watson bs4

     |████████████████████████████████| 368kB 17.7MB/s eta 0:00:01
     |████████████████████████████████| 204kB 6.3MB/s eta 0:00:01
     |████████████████████████████████| 112kB 28.5MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/ff/de/b5/79f9be24957fa20ee1a7e6ffa8a8e2a8da1ebfb44cac2789a1
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson bs4 ibm-cloud-sdk-core


In [3]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [4]:
recognition_service = SpeechToTextV1(IAMAuthenticator('X-bHu_VOFS_u9u39iEios-ltGw3ic_yKrODR6iVrXpPD'))
recognition_service.set_service_url('https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/8151ffe6-bb32-43a1-b201-4c79ea1ca613')
SPEECH_EXTENSION = "*.webm"
SPEECH_AUDIOTYPE = "audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

In [5]:
assistant = AssistantV2(version='2019-02-28', authenticator=IAMAuthenticator('X-bHu_VOFS_u9u39iEios-ltGw3ic_yKrODR6iVrXpPD'))
assistant.set_service_url('https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/8151ffe6-bb32-43a1-b201-4c79ea1ca613')
ASSISTANT_ID = "9744db38-ba96-4cd0-b075-54d161f0001b"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

In [6]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('X-bHu_VOFS_u9u39iEios-ltGw3ic_yKrODR6iVrXpPD'))
synthesis_service.set_service_url('https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/8151ffe6-bb32-43a1-b201-4c79ea1ca613')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)

In [ ]:
speak_text(message_assistant(recognize_audio()))